In [4]:
import numpy as np
import os
import glob
import torch
from data_loaders.dataloader3d import get_dataloader, load_data, MotionDataset

In [3]:
def subtract_root(data):
    #only after frames have been cut
    root = (data[0,8,:]+data[0, 9, :])/2
    return data - root

In [4]:
def drop_duplicate_frames(data):
    """
    Drop frames where all 25 rows are identical.
    
    Parameters:
    -----------
    data : numpy.ndarray
        Array with shape (frames, 25, 5)
    
    Returns:
    --------
    numpy.ndarray
        Filtered array with duplicate frames removed
    """
    first_row = data[:, 0:1, :]  # Shape: (frames, 1, 5)
    all_rows_same = np.all(data == first_row, axis=(1,2))

    mask = ~all_rows_same
    return data[mask]

In [5]:
data=np.load("data_example/700/vitpose_c1_a4_2/vitpose/keypoints_3d/smpl-keypoints-3d_cut.npy")

In [2]:
a,b = load_data("mydataset", split='train', keypointtype='6d')

In [3]:
dataset = MotionDataset(
    "gait",
    a,
    b,
    input_motion_length=296,
)

In [4]:
dataloader = get_dataloader(
    dataset, "train", batch_size=2, num_workers=2
)

In [5]:
print(len(dataset))        # number of samples
print(type(dataset))  

22
<class 'data_loaders.dataloader3d.MotionDataset'>


In [6]:
img, label = dataset[0]
print(img.shape, label.shape)  # shapes of the first sample

torch.Size([296, 135]) torch.Size([296, 135])


In [37]:
i=2
print(a[i].shape)
print(b[i].shape)

torch.Size([286, 72])
torch.Size([215, 72])


In [44]:
np.save("test.npy", a[2].numpy().reshape(-1, 24, 3))

In [8]:
minx=5000
maxx=-1000
miny=5000
maxy=-1000
minz=5000
maxz=-1000
root="observations"

for patient in os.listdir(root):
    for file in sorted(os.listdir(os.path.join(root, patient))):

        mypath=os.path.join(root, patient, file)
        mypath=os.path.join(mypath,"vitpose", "keypoints_3d", "smpl-keypoints-3d_cut.npy")
        print(mypath)
        data=np.load(mypath)
        print(data.shape)   
        data=drop_duplicate_frames(data)
        print(data.shape)
        datax=data[...,0]
        datay=data[...,1]
        dataz=data[...,2]
        data=subtract_root(data)

        curminx=np.min(datax)
        curmaxx=np.max(datax)
        curminy=np.min(datay)
        curmaxy=np.max(datay)
        curminz=np.min(dataz)
        curmaxz=np.max(dataz)

        if curminx < minx:
            minx = curminx
        if curmaxx > maxx:
            maxx = curmaxx
        if curminy < miny:
            miny = curminy
        if curmaxy > maxy:
            maxy = curmaxy
        if curminz < minz:
            minz = curminz
        if curmaxz > maxz:
            maxz = curmaxz
print("Overall min and max values:")
print(minx, maxx)
print(miny, maxy)
print(minz, maxz)

observations\753\vitpose_c1_a1\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(239, 25, 5)
(96, 25, 5)
observations\753\vitpose_c1_a1_2\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(239, 25, 5)
(98, 25, 5)
observations\753\vitpose_c1_a2\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(239, 25, 5)
(78, 25, 5)
observations\753\vitpose_c1_a2_2\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(240, 25, 5)
(70, 25, 5)
observations\753\vitpose_c1_a3\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(419, 25, 5)
(277, 25, 5)
observations\753\vitpose_c1_a3_2\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(420, 25, 5)
(195, 25, 5)
observations\753\vitpose_c1_a3_3\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(420, 25, 5)
(195, 25, 5)
observations\753\vitpose_c1_a4\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(240, 25, 5)
(122, 25, 5)
observations\753\vitpose_c1_a4_2\vitpose\keypoints_3d\smpl-keypoints-3d_cut.npy
(239, 25, 5)
(164, 25, 5)
observations\753\vitpose_c1_a5\vitpose\keypoints_3d\smpl-k

In [4]:
data = np.load('0\\fit-smplx\\smplx-params.npz')

print("Keys inside:", data.files)
for k in data.files:
    print(k, data[k].shape)

Keys inside: ['betas', 'global_orient', 'body_pose', 'transl', 'left_hand_pose', 'right_hand_pose', 'jaw_pose', 'leye_pose', 'reye_pose', 'expression']
betas (88, 11)
global_orient (88, 3)
body_pose (88, 63)
transl (88, 3)
left_hand_pose (88, 12)
right_hand_pose (88, 12)
jaw_pose (88, 3)
leye_pose (88, 3)
reye_pose (88, 3)
expression (88, 10)


In [3]:
data=np.load('0\\keypoints_3d\\smpl-keypoints-3d.npy')

In [5]:
data.shape

(88, 25, 5)

In [1]:
from utils.transformation_sixd import smplx_to_6d, sixd_to_smplx

In [7]:
x=smplx_to_6d("mydataset\\740\\20251002_c1_a1_Take1\\split_subjects\\0\\fit-smplx\\smplx-params.npz", "test/")

In [2]:
sixd_to_smplx("test/motion_6d_with_transl.npz", "test/", "smpl_models/")

c:\Users\Rui\Vorlesungskript\Master\Thesis\Thesis_project\utils\utils_transform.py:54: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Cross.cpp:67.)
  rot_vec_3 = torch.cross(rot_vec_1, rot_vec_2)


Performing forward kinematics to calculate joint positions...
Using generic SMPL-X model for forward kinematics.
Extracting joints for 109 frames...

Successfully reconstructed 3D keypoints.
Output shape: (109, 22, 3)
Saved result to: test/reconstructed_keypoints_3d_ref.npy


In [5]:
data=np.load("test/motion_6d_with_transl.npz")

In [6]:
data['transl'][60]

array([-0.8399562 ,  0.00595307, -1.8214412 ], dtype=float32)

In [7]:
data['transl'][50]

array([-0.66853815,  0.02578962, -1.5105556 ], dtype=float32)

In [8]:
t=torch.tensor(data['transl'])

In [10]:
t=t[...,:2]

In [11]:
t.shape

torch.Size([109, 2])

In [12]:
test=torch.randn(1, 296, 135)

In [13]:
test1=test[:,:, :3]
test2=test[:,:, 3:]
test2.shape

torch.Size([1, 296, 132])

In [34]:
loss_rot = torch.mean(
    torch.norm(
        (test2).reshape(-1, 6),
        2,
        1,
    )
)

loss_transl = torch.mean(
    torch.norm(
        (test1).reshape(-1, 3),
        2,
        1,
    )
)

In [35]:
print(loss_rot)
print(loss_transl)

tensor(2.3417)
tensor(1.6114)


In [36]:
loss_my = torch.mean(
    torch.norm(
        (test).reshape(-1, 135),
        2,
        1,
    )
)
loss_my

tensor(11.5594)

In [8]:
data['motion_6d'].shape

(110, 132)

In [14]:
'betas' in data

True